In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [29]:
# paso1: cargue la imagen y conviértala en una imagen en escala de grises
image = cv2.imread("1.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [30]:
# paso2: use el operador Sobel para calcular el gradiente en las direcciones x e y, y luego reste el gradiente en la dirección y en la dirección x. A través de esta resta, dejamos un área de imagen con un gradiente horizontal alto y un gradiente vertical bajo.
gradX = cv2.Sobel(gray, cv2.CV_32F, dx=1, dy=0, ksize=-1)
gradY = cv2.Sobel(gray, cv2.CV_32F, dx=0, dy=1, ksize=-1)

In [31]:
# subtract the y-gradient from the x-gradient
gradient = cv2.subtract(gradX, gradY)
gradient = cv2.convertScaleAbs(gradient)

In [5]:
# show image
cv2.imshow("first", gradient)
cv2.waitKey()

-1

In [32]:
# paso3: elimine el ruido en la imagen. Primero use un filtro de paso bajo para suavizar la imagen (núcleo de 9 x 9), lo que ayudará a suavizar el ruido de alta frecuencia en la imagen.
# El objetivo del filtro de paso bajo es reducir la tasa de cambio de la imagen. Por ejemplo, reemplace cada píxel con el valor promedio de los píxeles alrededor del píxel. Esto puede suavizar y reemplazar aquellas áreas donde la intensidad cambia significativamente.
# Luego, binarice la imagen borrosa. Cualquier píxel en la imagen de degradado no mayor de 90 se establece en 0 (negro). De lo contrario, el píxel se establece en 255 (blanco).
# blur and threshold the image
blurred = cv2.blur(gradient, (9, 9))
_, thresh = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)

In [7]:
# SHOW IMAGE
cv2.imshow("thresh", thresh)
cv2.waitKey()

-1

In [33]:
# paso4: En la imagen de arriba, vemos que hay muchas vacantes negras en el área del cuerpo de la abeja. Necesitamos llenar estas vacantes con blanco para que sea más fácil para el siguiente programa identificar el área del insecto.
# Esto requiere algunas operaciones morfológicas.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

In [21]:
# show image
cv2.imshow("closed1", closed)
cv2.waitKey()

-1

In [34]:
# paso5: De la imagen de arriba, descubrimos que todavía hay algunas pequeñas manchas blancas en la imagen, que interferirán con la detección de contornos de insectos en el futuro, y que deben eliminarse. Realizar corrosión morfológica y expansión 4 veces respectivamente.
# perform a series of erosions and dilations
closed = cv2.erode(closed, None, iterations=4)
closed = cv2.dilate(closed, None, iterations=4)

In [35]:
# show image
cv2.imshow("closed2", closed)
cv2.waitKey()

-1

In [36]:
(cnts, x) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
c = sorted(cnts, key=cv2.contourArea, reverse=True)[0]

In [37]:
# compute the rotated bounding box of the largest contour
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))

In [18]:
# draw a bounding box arounded the detected barcode and display the image
cv2.drawContours(image, [box], -1, (0, 255, 0), 3)
cv2.imshow("Image", image)
cv2.imwrite("contoursImage2.jpg", image)
cv2.waitKey(0)

-1

In [38]:
# step7: Recortar. En el cuadro se almacenan las coordenadas de los cuatro vértices del área rectangular verde. Recortaré la imagen del insecto como se muestra en el rectángulo rojo a continuación.
# Encuentra los valores máximos y mínimos de las coordenadas x e y de los cuatro vértices. La altura de la nueva imagen = maxY-minY, y el ancho = maxX-minX.
Xs = [i[0] for i in box]
Ys = [i[1] for i in box]
x1 = min(Xs)
x2 = max(Xs)
y1 = min(Ys)
y2 = max(Ys)
hight = y2 - y1
width = x2 - x1
cropImg = image[y1:y1+hight, x1:x1+width]

In [39]:
# show image
cv2.imshow("cropImg", cropImg)
cv2.imwrite("bee.jpg", cropImg)
cv2.waitKey()

-1